In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import linregress
from datetime import date, timedelta

In [ ]:
#download all data for ETHUSD from yahoo finance
ETH=yf.download("ETH-USD")

In [ ]:
#isolate the close data and turn it into a dataframe 
eth=pd.DataFrame(ETH.Close)
eth

In [ ]:
#calculate dialy returns 
eth["d_returns"]=np.log(eth.div(eth.shift(1)))
eth

In [ ]:
#get rid of the NaN data 
eth.dropna(inplace=True)
eth

In [ ]:
#calculate the entire returns of the daily returns column i.e. simulate a buy and hold strategy
eth.d_returns.sum()

In [ ]:
#calculate $1 invested on day 1 of the data i.e. 10-11-2017. 
#exp is a numpy fucntion for exponential
np.exp(eth.d_returns.sum())

In [ ]:
# create new column for cummulative returns 
eth["cumreturns"]=eth.d_returns.cumsum().apply(np.exp)
eth

In [ ]:
#plot cumreturns onto a graph 
eth.cumreturns.plot(figsize=(12,8),title="ETH buy and hold", fontsize=12)
plt.show()

In [ ]:
# calculate mean return annualised. as its crypto *365 
eth.d_returns.mean()*365

In [ ]:
#calculate standard deviation annualised
#standard deviation calculates how far it typically goes from the mean is the majoirty of the deviation from the mean
eth.d_returns.std()*np.sqrt(365)

In [ ]:
#To calculate drawdown by finding peak and trough - you calcualte the cumulative maximum 
eth["cummax"]=eth.cumreturns.cummax()
eth


In [ ]:
#plot just the cummreturns and cummax, which is plotting the highs
eth[["cumreturns","cummax"]].plot(figsize=(12,8),title="ETH buy and hold/cummax", fontsize=12)
plt.show()

In [ ]:
# calculate the drawdown by taking cummax and subtracting cumreturns. 
eth["drawdown"]=eth['cummax']-eth['cumreturns']
eth

In [ ]:
# calculate max drawdown 
eth.drawdown.max()

In [ ]:
#identify date of max drawdown
eth.drawdown.idxmax()

In [ ]:
#isoltate this day
eth.loc[(eth.index=='2022-06-18')]

In [ ]:
#percentage drawdown 
eth['drawdown%']=((eth['cummax']-eth['cumreturns'])/eth['cummax']*100)
eth

In [ ]:
#highest drawdonw from % 
eth['drawdown%'].max()

In [ ]:
#when did this take place?
eth['drawdown%'].idxmax()

In [ ]:
#identify locaiton of the drawdown and the lesd up to it  
eth.loc[(eth.index<='2018-12-14')]

### SMA Strategy 

In [ ]:
data = pd.DataFrame(eth.Close)
data

In [ ]:
#define the moving average 
sma_s=50
sma_l=100

In [ ]:
#create the moving average with rolling function
data["sma_s"]=data.Close.rolling(sma_s).mean()
data["sma_l"]=data.Close.rolling(sma_l).mean()
data

In [ ]:
#plot this
data.plot(figsize=(12,8), title="ETH - SMA{} | SMA{}".format(sma_s,sma_l), fontsize=12)

In [ ]:
#plot a specific year
data.loc["2018"].plot(figsize=(12,8), title="ETH - SMA{} | SMA{}".format(sma_s,sma_l), fontsize=12)

In [ ]:
#get rid of any NaNs
data.dropna(inplace=True)
data

In [ ]:
#strategy sma_s > sma_l = long or sma_s < sma_l = short 
data["position"]=np.where(data["sma_s"]>data["sma_l"],1,-1)
data

In [ ]:
#plot when we are long or short plotting 1 to -1 on a secondary y axis. we have also .loc'd 2018
data.loc["2019", ["sma_s", "sma_l", "position"]].plot(figsize=(12,8), title="ETH - SMA{} | SMA{}".format(sma_s,sma_l), fontsize=12,secondary_y="position")

In [ ]:
#calculate buyandhold returns
data["buy&hold"]=np.log(data.Close.div(data.Close.shift(1)))
data

In [ ]:
#returns from sma strat
data["strategy"]=data["buy&hold"]*data.position.shift(1)
data

In [ ]:
 
data.dropna(inplace=True)


In [ ]:
#compare the 2 strategies
data[["buy&hold","strategy"]].sum()

In [ ]:
#calculate absolute performance what $1 investment would be 
data[["buy&hold","strategy"]].sum().apply(np.exp)

In [ ]:
#annual std
data[["buy&hold","strategy"]].std()*np.sqrt(365)

In [ ]:
data.Close.plot(figsize=(12,8))

### Strategy adjusted to long bias 

In [ ]:
data["position2"]=np.where(data["sma_s"]>data["sma_l"],1,0)

In [ ]:
data["strategy2"]=data["buy&hold"]*data.position2.shift(1)

In [ ]:
data

In [ ]:
data.dropna(inplace=True)

In [ ]:
#compare both strats 
data[["buy&hold", "strategy", "strategy2"]].sum()

In [ ]:
# with $1
data[["buy&hold", "strategy", "strategy2"]].sum().apply(np.exp)

In [ ]:
data[["buy&hold","strategy","strategy2"]].std()*np.sqrt(365)

##### L/S strategy has been better than long only and buy&hold